In [7]:
import requests
import json
import os
import time
outputpath="/content/test.json"
f = open("/content/quotestable.json",encoding='utf-8')

# returns JSON object as
# a dictionary
data1 = json.load(f)
# os.remove(outputpath)
f = open(outputpath, "a",encoding="utf-8")
idcount=0
counter=0
list1=[]
#clean the dataset of duplicate authors
authorset=set()
for count,i in enumerate(data1):
  authorset.add(i["author"].strip())
print(len(authorset))
for auth in authorset:
  url = 'https://query.wikidata.org/sparql'
  query = '''
  SELECT ?item ?prefLabel ?birthLocation ?birthLocationLabel ?occupation ?occupationLabel ?coord ?region ?regionLabel WHERE {
        VALUES ?prefLabel {  "'''+auth+'''"@en }
        ?item rdfs:label|skos:altLabel ?prefLabel;
             wdt:P31 wd:Q5;
             wdt:P19 ?birthLocation.
             OPTIONAL {?birthLocation wdt:P625 ?coord. }
             OPTIONAL {?birthLocation wdt:P17 ?region. }
             OPTIONAL { ?item wdt:P106 ?occupation . }
             
SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
  '''
  time.sleep(1)
  try:
    r = requests.get(url, params = {'format': 'json', 'query': query}, timeout=5)
    data = r.json()
  except Exception as e:
    print(e)
    exit(1)
  try:
    a =data["results"]['bindings'][-1]["birthLocationLabel"]["value"]
    
  except:
    continue


  authordict={}
  authordict["name"] = auth
  authordict["birthlocation"] = a
  if "coord" in data["results"]['bindings'][-1]:
    authordict["coords"] =  data["results"]['bindings'][-1]["coord"]["value"]
  else:
    authordict["coords"] = None
  if "regionLabel" in data["results"]['bindings'][-1]:
    authordict["regionLabel"] =  data["results"]['bindings'][-1]["regionLabel"]["value"]
  else:
    authordict["coords"] = None
  jobs=[]
  for x in data["results"]['bindings']:
    if "occupationLabel" in x:
      jobs.append(x["occupationLabel"]["value"])
  authordict["occupation"] = jobs
  print(str(authordict) + ",")
  list1.append(authordict)
with open("authors.json", 'w') as f2:
    json.dump(list1, f2)
print(list1)

5288
{'name': 'Robert Frost', 'birthlocation': 'North Elmham', 'coords': 'Point(0.939719444 52.748561111)', 'regionLabel': 'United Kingdom', 'occupation': ['historian', 'writer', 'poet', 'playwright', 'pedagogue', 'cricketer']},
{'name': 'Bonnie Tyler', 'birthlocation': 'Skewen', 'coords': 'Point(-3.8399 51.6609)', 'regionLabel': 'United Kingdom', 'occupation': ['singer', 'singer-songwriter', 'songwriter', 'recording artist']},
{'name': 'Charles Churchill', 'birthlocation': 'Devon', 'coords': 'Point(-3.8 50.7)', 'regionLabel': 'United Kingdom', 'occupation': ['writer', 'poet', 'military personnel', 'politician', 'military officer']},
{'name': 'Seth Brown', 'birthlocation': 'Klamath Falls', 'coords': 'Point(-121.7775 42.223333333)', 'regionLabel': 'United States of America', 'occupation': ['baseball player']},
{'name': 'William Jennings Bryan', 'birthlocation': 'Salem', 'coords': 'Point(-88.9456158 38.6269929)', 'regionLabel': 'United States of America', 'occupation': ['writer', 'lawyer